In [2]:
# Setting the environment variables
import os
import sys
import pandas as pd
import numpy as np
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook --no-browser"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

# Ecommerce Churn Assignment

The aim of the assignment is to build a model that predicts whether a person purchases an item after it has been added to the cart or not. Being a classification problem, you are expected to use your understanding of all the three models covered till now. You must select the most robust model and provide a solution that predicts the churn in the most suitable manner. 

For this assignment, you are provided the data associated with an e-commerce company for the month of October 2019. Your task is to first analyse the data, and then perform multiple steps towards the model building process.

The broad tasks are:
- Data Exploration
- Feature Engineering
- Model Selection
- Model Inference

### Data description

The dataset stores the information of a customer session on the e-commerce platform. It records the activity and the associated parameters with it.

- **event_time**: Date and time when user accesses the platform
- **event_type**: Action performed by the customer
            - View
            - Cart
            - Purchase
            - Remove from cart
- **product_id**: Unique number to identify the product in the event
- **category_id**: Unique number to identify the category of the product
- **category_code**: Stores primary and secondary categories of the product
- **brand**: Brand associated with the product
- **price**: Price of the product
- **user_id**: Unique ID for a customer
- **user_session**: Session ID for a user


### Initialising the SparkSession

The dataset provided is 5 GBs in size. Therefore, it is expected that you increase the driver memory to a greater number. You can refer to notebook 1 for the steps involved here.

In [3]:
# Loading the clean data
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct ,col, avg, mean, isnan, when, count, col, to_timestamp
from pyspark.sql.types import IntegerType

In [4]:
# initialising the session with 14 GB driver memory
MAX_MEMORY = "14G"

spark = SparkSession \
    .builder \
    .appName("demo") \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

spark

In [5]:
#Reading the previously stored CSV file
df_lr = spark.read.csv('/home/ec2-user/df_updated2', header= False, inferSchema= True)

In [6]:
df_lr.show(15)

+--------+---+---+-----------+-----------+---+---+---+-----+---+----+-------+----+----+
|     _c0|_c1|_c2|        _c3|        _c4|_c5|_c6|_c7|  _c8|_c9|_c10|   _c11|_c12|_c13|
+--------+---+---+-----------+-----------+---+---+---+-----+---+----+-------+----+----+
|    view|540|  6|electronics| smartphone| 35|  1| 35| null| 35| 3.0| huawei|   0|   0|
|    view|114|  5| appliances|environment|  1|  1|  1| null|  4| 0.0| others|   0|   0|
|    view| 39|  3|electronics|     clocks|  2|  2|  2| null|  2| 3.0|  casio|   0|   0|
|    view|167|  6|electronics|      audio|  2|  2|  2| null| 14| 2.0| others|   0|   0|
|    view|161|  6|electronics|      audio| 12|  1|  1| null|121| 2.0|  apple|   0|   0|
|    view|437|  4|  computers| components|  4|  1| 29| null| 85| 2.0| others|   0|   0|
|    view| 45|  5|    apparel|      shoes| 11|  2| 11| null|330| 0.0| others|   0|   0|
|    view|243|  3|electronics| smartphone|  2|  1|  2| null|  2| 1.0|samsung|   0|   0|
|    view|146|  2|electronics|  

In [7]:
df_lr.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)



In [8]:
#Dropping all null values
df_lr=df_lr.na.drop()

In [10]:
#Renaming the column names in line with the updates
df_lr = df_lr.select(col("_c0").alias("event_type"), col("_c1").alias("price"), col("_c2").alias("day_of_week"), col("_c3").alias("category1"), col("_c4").alias("category2"), col("_c5").alias("activity_countval"), col("_c6").alias("product_view_counts"), col("_c7").alias("category2_view_counts"), col("_c8").alias("average_shopping_expense"), col("_c9").alias("session_counts"), col("_c10").alias("binnedhour"), col("_c11").alias("brand_new"), col("_c12").alias("is_purchased"), col("_c13").alias("label"))

In [11]:
df_lr.show(5)

+----------+-----+-----------+-----------+-----------+-----------------+-------------------+---------------------+------------------------+--------------+----------+---------+------------+-----+
|event_type|price|day_of_week|  category1|  category2|activity_countval|product_view_counts|category2_view_counts|average_shopping_expense|session_counts|binnedhour|brand_new|is_purchased|label|
+----------+-----+-----------+-----------+-----------+-----------------+-------------------+---------------------+------------------------+--------------+----------+---------+------------+-----+
|  purchase|  179|          2|electronics| smartphone|                4|                  4|                    4|                   192.0|             7|       2.0|     sony|           1|    1|
|      view|  197|          7|electronics| smartphone|               13|                  3|                  155|                  228.25|           231|       2.0|   xiaomi|           0|    0|
|      view|  203|       

<hr>

## Task 3: Model Selection
3 models for classification:	
- Logistic Regression
- Decision Tree
- Random Forest

### Model 1: Logistic Regression

In [13]:
# Additional steps for Logistic regression - Feature selection, Correlation, etc.
df_lr=df_lr.withColumn("is_purchased",df_lr["is_purchased"].cast(IntegerType()))

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#### Feature Transformation

In [15]:
# Check if only the required columns are present to build the model
# If not, drop the redundant columns
from pyspark.sql.functions import when, count, col, isnull, isnan
df_lr.select([count(when(isnan(c), c)).alias(c) for c in df_lr.columns]).show()

+----------+-----+-----------+---------+---------+-----------------+-------------------+---------------------+------------------------+--------------+----------+---------+------------+-----+
|event_type|price|day_of_week|category1|category2|activity_countval|product_view_counts|category2_view_counts|average_shopping_expense|session_counts|binnedhour|brand_new|is_purchased|label|
+----------+-----+-----------+---------+---------+-----------------+-------------------+---------------------+------------------------+--------------+----------+---------+------------+-----+
|         0|    0|          0|        0|        0|                0|                  0|                    0|                       0|             0|         0|        0|           0|    0|
+----------+-----+-----------+---------+---------+-----------------+-------------------+---------------------+------------------------+--------------+----------+---------+------------+-----+



In [16]:
!pip3 install xverse

Defaulting to user installation because normal site-packages is not writeable


In [17]:
df_lr.groupBy('binnedhour').count().orderBy('count', ascending= False).show(5)

+----------+-------+
|binnedhour|  count|
+----------+-------+
|       1.0|1445390|
|       2.0|1260359|
|       0.0| 705528|
|       3.0| 391342|
+----------+-------+



In [18]:
df_lr.count()

3802619

In [19]:
df_lr.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- category1: string (nullable = true)
 |-- category2: string (nullable = true)
 |-- activity_countval: integer (nullable = true)
 |-- product_view_counts: integer (nullable = true)
 |-- category2_view_counts: integer (nullable = true)
 |-- average_shopping_expense: double (nullable = true)
 |-- session_counts: integer (nullable = true)
 |-- binnedhour: double (nullable = true)
 |-- brand_new: string (nullable = true)
 |-- is_purchased: integer (nullable = true)
 |-- label: integer (nullable = true)



In [20]:
# Categorising the attributes into its type - Continuous and Categorical
categorical_features=['brand_new','category2', 'day_of_week']
continuous_features=['price', 'product_view_counts', 'category2_view_counts', 'average_shopping_expense', 'session_counts', 'activity_countval']
output_label = 'is_purchased'

In [21]:
# Feature transformation for categorical features
#Using StringIndexer
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler 

In [22]:
stages=[]

In [23]:
for categoricalCol in categorical_features:
    stringIndexer=StringIndexer(inputCol=categoricalCol,outputCol=categoricalCol+'_index').setHandleInvalid("keep")
    encoder=OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()],outputCols=\
                                   [categoricalCol+"_enc"])
    stages+=[stringIndexer,encoder]

In [24]:
assemblerInputs=[col+"_enc" for col in categorical_features]+continuous_features
assembler=VectorAssembler(inputCols=assemblerInputs,outputCol="features")
stages+=[assembler]

In [25]:
# Vector assembler to combine all the features
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
pipeline=Pipeline(stages=stages)

In [27]:
# Transforming the dataframe df
df_lr_tf = pipeline.fit(df_lr).transform(df_lr)

In [28]:
# Schema of the transformed df
df_lr_tf.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- category1: string (nullable = true)
 |-- category2: string (nullable = true)
 |-- activity_countval: integer (nullable = true)
 |-- product_view_counts: integer (nullable = true)
 |-- category2_view_counts: integer (nullable = true)
 |-- average_shopping_expense: double (nullable = true)
 |-- session_counts: integer (nullable = true)
 |-- binnedhour: double (nullable = true)
 |-- brand_new: string (nullable = true)
 |-- is_purchased: integer (nullable = true)
 |-- label: integer (nullable = true)
 |-- brand_new_index: double (nullable = false)
 |-- brand_new_enc: vector (nullable = true)
 |-- category2_index: double (nullable = false)
 |-- category2_enc: vector (nullable = true)
 |-- day_of_week_index: double (nullable = false)
 |-- day_of_week_enc: vector (nullable = true)
 |-- features: vector (nullable = true)



In [29]:
# Checking the elements of the transformed df - Top 20 rows
df_lr_tf.show(20)

+----------+-----+-----------+-----------+-----------+-----------------+-------------------+---------------------+------------------------+--------------+----------+---------+------------+-----+---------------+---------------+---------------+---------------+-----------------+---------------+--------------------+
|event_type|price|day_of_week|  category1|  category2|activity_countval|product_view_counts|category2_view_counts|average_shopping_expense|session_counts|binnedhour|brand_new|is_purchased|label|brand_new_index|  brand_new_enc|category2_index|  category2_enc|day_of_week_index|day_of_week_enc|            features|
+----------+-----+-----------+-----------+-----------+-----------------+-------------------+---------------------+------------------------+--------------+----------+---------+------------+-----+---------------+---------------+---------------+---------------+-----------------+---------------+--------------------+
|  purchase|  179|          2|electronics| smartphone|    

In [31]:
# Scaling the features vector
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler().setInputCol("features").setOutputCol("scaled_features")
df_lrr = scaler.fit(df_lr_tf).transform(df_lr_tf)

In [32]:
df_lrr.select("features","scaled_features").head()

Row(features=SparseVector(90, {16: 1.0, 21: 1.0, 83: 1.0, 84: 179.0, 85: 4.0, 86: 4.0, 87: 192.0, 88: 7.0, 89: 4.0}), scaled_features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0695, 0.0129, 0.0015, 0.0746, 0.0021, 0.0058]))

#### Train-test split

In [33]:
# Splitting the data into train and test (Remember you are expected to compare the model later)
traindata, testdata = df_lrr.randomSplit([0.7,0.3], seed=100)

In [34]:
#Importing Logistic Regression
from pyspark.ml.classification import LogisticRegression

# create the logistic model, configure it and call fit it to the data. 
lr = LogisticRegression(featuresCol='scaled_features', labelCol='label')

In [35]:
traindata.select("features").show(10)

+--------------------+
|            features|
+--------------------+
|(90,[1,31,82,84,8...|
|(90,[1,31,82,84,8...|
|(90,[1,31,82,84,8...|
|(90,[16,23,81,84,...|
|(90,[1,31,79,84,8...|
|(90,[1,31,79,84,8...|
|(90,[3,23,81,84,8...|
|(90,[3,23,81,84,8...|
|(90,[0,23,81,84,8...|
|(90,[3,23,81,84,8...|
+--------------------+
only showing top 10 rows



In [36]:
traindata.select("scaled_features").show(10)

+--------------------+
|     scaled_features|
+--------------------+
|[0.0,1.0,0.0,0.0,...|
|[0.0,1.0,0.0,0.0,...|
|[0.0,1.0,0.0,0.0,...|
|[0.0,0.0,0.0,0.0,...|
|[0.0,1.0,0.0,0.0,...|
|[0.0,1.0,0.0,0.0,...|
|[0.0,0.0,0.0,1.0,...|
|[0.0,0.0,0.0,1.0,...|
|[1.0,0.0,0.0,0.0,...|
|[0.0,0.0,0.0,1.0,...|
+--------------------+
only showing top 10 rows



In [37]:
traindata.select("label").show(10)

+-----+
|label|
+-----+
|    0|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
|    1|
+-----+
only showing top 10 rows



#### Model Fitting

In [39]:
# Building the model
model = lr.fit(traindata)

In [40]:
# Fitting the model on transformed df
result = model.evaluate(df_lrr)

#### Model Analysis

Required Steps:
- Fit on test data
- Performance analysis
    - Appropriate Metric with reasoning
    - Threshold selection

In [41]:
df_lrr.show(20)

+----------+-----+-----------+-----------+-----------+-----------------+-------------------+---------------------+------------------------+--------------+----------+---------+------------+-----+---------------+---------------+---------------+---------------+-----------------+---------------+--------------------+--------------------+
|event_type|price|day_of_week|  category1|  category2|activity_countval|product_view_counts|category2_view_counts|average_shopping_expense|session_counts|binnedhour|brand_new|is_purchased|label|brand_new_index|  brand_new_enc|category2_index|  category2_enc|day_of_week_index|day_of_week_enc|            features|     scaled_features|
+----------+-----+-----------+-----------+-----------+-----------------+-------------------+---------------------+------------------------+--------------+----------+---------+------------+-----+---------------+---------------+---------------+---------------+-----------------+---------------+--------------------+-----------------

In [42]:
result.accuracy

0.8116329824260595

In [43]:
result.recallByLabel

[0.8515713674881443, 0.7645165857570395]

In [44]:
result.precisionByLabel

[0.8101101408330914, 0.8136428163130099]

In [45]:
result.fMeasureByLabel()

[0.8303234970005865, 0.7883150803872394]

In [46]:
result.areaUnderROC

0.8854948369999165

#### Updated model, if any
Repeat the steps 